# Generating random poems with Python #


<div style="text-align:center;margin-top:40px">(I never said they would be good poems)</div>

## Phone autocomplete ##

You can generate random text that sounds like you with your smartphone keyboard:

<div style="float:left">![Smartphone keyboard](images/phone_keyboard.png)</div>
<div style="float:right">![Smartphone_autocomplete](images/phone_autocomplete.gif)</div>

## So, how does it work? ##

First, we need a **corpus**, or the text our generator will recombine into new sentences:

In [1]:
corpus = 'The quick brown fox jumps over the lazy dog'

Simplest word **tokenization** is to split on spaces:

In [2]:
words = corpus.split(' ')
words

['The', 'quick', 'brown', 'fox', 'jumps', 'over', 'the', 'lazy', 'dog']

To create **bigrams**, iterate through the list of words with two indicies, one of which is offset by one:

In [3]:
bigrams = [b for b in zip(words[:-1], words[1:])]
bigrams

[('The', 'quick'),
 ('quick', 'brown'),
 ('brown', 'fox'),
 ('fox', 'jumps'),
 ('jumps', 'over'),
 ('over', 'the'),
 ('the', 'lazy'),
 ('lazy', 'dog')]

How do we use the bigrams to predict the next word given the first word?

 Return every second element where the first element matches the **condition**:

In [4]:
condition = 'the'
next_words = [bigram[1] for bigram in bigrams
              if bigram[0].lower() == condition]
next_words

['quick', 'lazy']

(<font color="blue">The</font> <font color="red">quick</font>) (quick brown) ... (<font color="blue">the</font> <font color="red">lazy</font>) (lazy dog)

Either “<font color="red">quick</font>” or “<font color="red">lazy</font>” could be the next word.

## Trigrams and Ngrams ##

We can partition by threes too:

(<font color="blue">The</font> <font color="red">quick brown</font>) (quick brown fox) ... (<font color="blue">the</font> <font color="red">lazy dog</font>)


Or, the condition can be two words (`condition = 'the lazy'`):

(The quick brown) (quick brown fox) ... (<font color="blue">the lazy</font> <font color="red">dog</font>)


These are **trigrams**.

We can partition any **N** number of words together as **ngrams**.

So earlier we got:

In [5]:
next_words

['quick', 'lazy']

How do we know which one to pick as the next word?

Why not the word that occurred the most often after the condition in the corpus?

We can use a **Conditional Frequency Distribution (CFD)** to figure that out!

A **CFD** can tell us: given a **condition**, what is **likely** to follow?

## Conditional Frequency Distributions (CFDs) ##

In [6]:
words = 'The quick brown fox jumped over the lazy dog and the quick cat'.split(' ')
print words

['The', 'quick', 'brown', 'fox', 'jumped', 'over', 'the', 'lazy', 'dog', 'and', 'the', 'quick', 'cat']


In [7]:
from collections import defaultdict

cfd = defaultdict(lambda: defaultdict(lambda: 0))
condition = 'the'

In [8]:
for i in range(len(words) - 2):
    if words[i].lower() == condition:
        cfd[condition][words[i+1]] += 1

# pretty print the defaultdict 
{k: dict(v) for k, v in dict(cfd).items()}

{'the': {'lazy': 1, 'quick': 2}}

## What's the most likely? ##

In [9]:
max(cfd[condition])

'quick'

## Whole sentences can be the conditions and values too ##

Which is basically the way cleverbot works:

![Cleverbot](images/cleverbot.png)

[http://www.cleverbot.com/](http://www.cleverbot.com/)

## Random text! ##

In [10]:
import nltk
import random

TEXT = nltk.corpus.gutenberg.words('austen-emma.txt')

# NLTK shortcuts :)
bigrams = nltk.bigrams(TEXT)
cfd = nltk.ConditionalFreqDist(bigrams)

# pick a random word from the corpus to start with
word = random.choice(TEXT)
# generate 15 more words
for i in range(15):
    print word,
    if word in cfd:
        word = random.choice(cfd[word].keys())
    else:
        break

must therefore that half ago for hope that occasion , Perry -- abundance about ten


## Random poems ##

Generating random poems is simply limiting the choice of the next word by some constraint:

* words that rhyme with the previous line
* words that match a certain syllable count
* words that alliterate with words on the same line
* etc.

![Buzzfeed Haiku Generator](images/buzzfeed.png)

[http://mule.hallada.net/nlp/buzzfeed-haiku-generator/](http://mule.hallada.net/nlp/buzzfeed-haiku-generator/)

## Remember these? ##

![madlibs](images/madlibs.png)

## Madlibs ##

These worked so well because they forced the random words (chosed by you) to fit into the syntactical structure and parts-of-speech of an existing sentence.

You end up with **syntactically** correct sentences that are **semantically** random.

We can do the same thing!

## NLTK Syntax Trees! ##

In [11]:
from stat_parser import Parser
parser = Parser()
print parser.parse('The quick brown fox jumps over the lazy dog.')

(S
  (NP (DT the) (NN quick))
  (VP
    (VB brown)
    (NP
      (NP (JJ fox) (NN jumps))
      (PP (IN over) (NP (DT the) (JJ lazy) (NN dog)))))
  (. .))


## Swaping matching syntax subtrees between two corpora ##

In [15]:
from syntax_aware_generate import generate

# inserts matching syntax subtrees from trump.txt into
# trees from austen-emma.txt
generate('trump.txt', word_limit=15)

(SBARQ
  (SQ
    (NP (PRP she))
    (VP
      (VBD was)
      (VBN obliged)
      (S+VP (TO to) (VP (VB stop) (CC and) (VB think)))))
  (. .))
she was obliged to stop and think .
They was hacked to amp ; support !
(SBARQ
  (SQ
    (NP (PRP They))
    (VP
      (VBD was)
      (VBN hacked)
      (S+VP (TO to) (VP (VB amp) (CC ;) (VB support)))))
  (. !))


## spaCy ##

![spaCy speed comparison](images/spacy_speed.png)

[https://spacy.io/docs/api/#speed-comparison](https://spacy.io/docs/api/#speed-comparison)

## Character-based Recurrent Neural Networks ##

![RNN Paper](images/rnn_paper.png)

[http://www.cs.utoronto.ca/~ilya/pubs/2011/LANG-RNN.pdf](http://www.cs.utoronto.ca/~ilya/pubs/2011/LANG-RNN.pdf)

## Implementation: char-rnn ##

![char-rnn](images/char-rnn.png)

[https://github.com/karpathy/char-rnn](https://github.com/karpathy/char-rnn)

## Generating Shakespeare with char-rnn ##

![Shakespeare](images/shakespeare.png)

[http://karpathy.github.io/2015/05/21/rnn-effectiveness/](http://karpathy.github.io/2015/05/21/rnn-effectiveness/)

# The end #

Questions?